<a href="https://colab.research.google.com/github/books-by-chansung/mh-mlops-model/blob/main/notebooks/tb_tf_flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
ds, info = tfds.load('tf_flowers',
                     split=['train[:80%]', 
                            'train[80%:90%]', 
                            'train[90%:]'],
                     with_info=True)

ds_train, ds_val, ds_test = ds[0], ds[1], ds[2]

In [ ]:
input_shape=(224, 224, 3)
num_classes = info.features['label'].num_classes

In [ ]:
import tensorflow as tf

def normalize_img(data):
  data['image'] = tf.image.resize(data['image'], [224, 224])
  return tf.cast(data['image'], tf.float32) / 255., data['label']

In [ ]:
ds_train = ds_train.map(
    normalize_img, 
    num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(len(ds_train))
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_val = ds_val.map(
    normalize_img, 
    num_parallel_calls=tf.data.AUTOTUNE)
ds_val = ds_val.batch(128)
ds_val = ds_val.cache()
ds_val = ds_val.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, 
    num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
from tensorboard.plugins.hparams import api as hp

In [ ]:
glogdir='hparam_tuning_tf_flower'

In [ ]:
HP_LR = hp.HParam('learning_rate', hp.Discrete([0.1, 0.01, 0.001]))
HP_DEPTH = hp.HParam('depth', hp.Discrete([0, 1, 2, 3]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(f'logs/{glogdir}').as_default():
  hp.hparams_config(
    hparams=[HP_LR, HP_DEPTH],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
import tensorflow_hub as hub
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.losses import SparseCategoricalCrossentropy

def train_test_model(input_shape, logdir, hparams):
    model_uri = "https://tfhub.dev/google/imagenet/resnet_v1_50/feature_vector/5"

    base_model = hub.KerasLayer(model_uri)
    base_model.trainable=False

    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    for depth_num in range(hparams[HP_DEPTH]):
        x = Dense(1024)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)

    model.compile(optimizer=Adam(learning_rate=hparams[HP_LR]),
                  loss=SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    early_stop_callback = EarlyStopping(patience=3)

    model.fit(
        ds_train,
        epochs=20,
        validation_data=ds_val,
        callbacks=[early_stop_callback, 
                   TensorBoard(logdir, histogram_freq=1), 
                   hp.KerasCallback(logdir, hparams)]
    )
    _, accuracy = model.evaluate(ds_test)
    return accuracy

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(input_shape, run_dir, hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0

for lr in HP_LR.domain.values:
  for depth in HP_DEPTH.domain.values:
      hparams = {
          HP_LR: lr,
          HP_DEPTH: depth,
      }    
      run_name = f'run-{session_num}'
      print(f'--- Starting trial: {run_name}')
      print({h.name: hparams[h] for h in hparams})
      run(f'logs/{glogdir}/{run_name}', hparams)
      session_num += 1

# for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):

--- Starting trial: run-0
{'learning_rate': 0.001, 'depth': 0}
Epoch 1/20
23/23 [==============================] - 26s 930ms/step - loss: 0.8694 - accuracy: 0.6826 - val_loss: 0.4282 - val_accuracy: 0.8447
Epoch 2/20
23/23 [==============================] - 21s 903ms/step - loss: 0.4058 - accuracy: 0.8634 - val_loss: 0.3312 - val_accuracy: 0.8965
Epoch 3/20
23/23 [==============================] - 21s 919ms/step - loss: 0.2995 - accuracy: 0.9050 - val_loss: 0.3118 - val_accuracy: 0.8937
Epoch 4/20
23/23 [==============================] - 21s 921ms/step - loss: 0.2488 - accuracy: 0.9244 - val_loss: 0.2792 - val_accuracy: 0.9074
Epoch 5/20
23/23 [==============================] - 21s 916ms/step - loss: 0.2114 - accuracy: 0.9401 - val_loss: 0.2656 - val_accuracy: 0.9155
Epoch 6/20
23/23 [==============================] - 23s 995ms/step - loss: 0.1839 - accuracy: 0.9520 - val_loss: 0.2561 - val_accuracy: 0.9128
Epoch 7/20
23/23 [==============================] - 23s 1s/step - loss: 0.1631 

In [ ]:
!tensorboard dev upload --logdir ./logs/{glogdir} \
  --name "꽃 데이터셋 분류 실험" \
  --one_shot


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./logs/{glogdir}

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=VARJ763lL5IO2W99v3jlDO4uLHDmuL&prompt=consent&access_type=offl